In [1]:
# | default_exp solvemodels

## Setup

In [2]:
# | export


import json
import sys

import cobra

# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.buildingediting
import mmon_gcm.solving
import mmon_gcm.supermodel
import pandas as pd

In [3]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster
# This is where to adjust whether fva is run if running in notebook
args = {
    "run_fva": "True",
    "no_processes": 6,
    "output_dir": "../outputs/model_solutions/",
    "model_file": "../models/4_stage_GC.xml",
    "map_file": "../inputs/map.json",
    "parameters_file": "../inputs/arabidopsis_parameters.csv",
}

sys.argv = ["script_name"] + list(args.values())

In [4]:
# | export

run_fva = sys.argv[1]
if run_fva == "True":
    run_fva = True
elif run_fva == "False":
    run_fva = False
else:
    raise ValueError(f"Please specify True or False for run_fva, not {run_fva}")

no_processes = int(sys.argv[2])
output_dir = sys.argv[3]
model_file = sys.argv[4]
map_file = sys.argv[5]
parameters_file = sys.argv[6]

## Instantiate and apply base constraints to model

### Import FBA model

In [5]:
# | export

four_stage_GC_model = cobra.io.sbml.read_sbml_model(model_file)  # read model

No objective coefficients in model. Unclear what should be optimized


Decrease tolerance

In [6]:
# | export

print(four_stage_GC_model.solver.configuration.tolerances.integrality)
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)
four_stage_GC_model.solver.configuration.tolerances.feasibility = 1e-7
print(four_stage_GC_model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-07


### Define reactions to use for FVA

In [7]:
# | export

if run_fva == True:
    linker_list = [
        reaction
        for reaction in four_stage_GC_model.reactions
        if "gc_Linker_2" in reaction.id
        or "gc_Linker_1" in reaction.id
        or "_total_pseudolinker_1" in reaction.id
        or "_total_pseudolinker_2" in reaction.id
        or "ae_gc" in reaction.id
    ]

    extra_fva_rxns = set()

    with open(map_file, "r+") as f:
        map_data = json.load(f)

    for reaction in map_data[1]["reactions"].values():
        extra_fva_rxns.add(four_stage_GC_model.reactions.get_by_id(reaction["bigg_id"]))

    fva_list = list(set(linker_list) | extra_fva_rxns)

else:
    fva_list = []

### Instantiate Supermodel

Import parameters from csv, created in [parameters notebook](1.2_defining_parameters.ipynb)

In [8]:
# | export

parameters_df = pd.read_csv(parameters_file, index_col=0)
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
R_ch_vol,2.004760e-01,Dimensionless,Knoblauch et al. (2023)
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)


In [9]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(parameters_df.loc[:, "Value"], fba_model=four_stage_GC_model)

In [10]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True);

Volume in phase 0 is 0.38pL
Volume in phase 1 is 0.4375pL
Volume in phase 2 is 0.38pL
Volume in phase 3 is 0.38pL


In [11]:
# | export

arabidopsis_supermodel.get_volumes(printouts=True, per_guard_cell=False);

Volume in phase 0 is 0.0002204dm3
Volume in phase 1 is 0.00025375dm3
Volume in phase 2 is 0.0002204dm3
Volume in phase 3 is 0.0002204dm3


This is a little different to Jezek and Blatt (2017), where they have closed as 0.3-0.4pL and open as 0.5-0.65. Open apertures are on the small side but still within their range (2-6). Wang used a volume of 0.4pL.

### Constrain SuperModel

#### Constrain osmolarity of the model using the equation from the OnGuard model (Hills et al 2012)

In [12]:
# | export

arabidopsis_supermodel.constrain_osmolarity(printouts=True);

Raw osmolarities: [0.039359327616195004, 0.05492236409486689, 0.039359327616195004, 0.039359327616195004]
Osmolarities in mM: [178.58134127 216.44281417 178.58134127 178.58134127]
Change in osmolarity: 37.86147289566091mM
c osmolarities constrained to [0.00980047 0.01367567 0.00980047 0.00980047]
v osmolarities constrained to [0.02955886 0.0412467  0.02955886 0.02955886]


In [13]:
import numpy as np

In [14]:
np.array(arabidopsis_supermodel.get_osmolarities())

array([0.03935933, 0.05492236, 0.03935933, 0.03935933])

In [15]:
help(arabidopsis_supermodel.get_osmolarities)

Help on method get_osmolarities in module mmon_gcm.supermodel:

get_osmolarities(apertures='default', equation='onguard', printouts=False) method of mmon_gcm.supermodel.SuperModel instance
    Calculates the osmolarities at each phase of the model using the equation
    specified, default is onguard. Adds these as an attribute to the SuperModel



In [16]:
mmoles_per_m2 = np.array(arabidopsis_supermodel.get_osmolarities())
mmoles_per_gc = mmoles_per_m2 / arabidopsis_supermodel.N_gcs
moles_per_gc = mmoles_per_gc * 10**-3
fmoles_per_gc = moles_per_gc / 10**-15
print(f"Closed fmoles: {fmoles_per_gc[0]}")
print(f"Open fmoles: {fmoles_per_gc[1]}")
print(f"Increase in fmoles: {fmoles_per_gc[1] - fmoles_per_gc[0]}")

Closed fmoles: 67.86090968309483
Open fmoles: 94.69373119804636
Increase in fmoles: 26.832821514951533


#### Constrain photons using a PPFD of 150µmolm$^{-2}$s$^{-1}$, same as used in Horrer et al (2016)

In [17]:
# | export

PPFD = 150
arabidopsis_supermodel.constrain_photons(PPFD, printouts=True);

Total leaf volume: 0.17dm3
Guard cell volume in 1m2 leaf: 0.0003dm3
Mesophyll cell volume in 1m2 leaf: 0.091dm3
Proportion of the leaf that is gc: 0.003
PPFD 150umolphotonsm-2
Photon influx into leaf: 486.0 mmolphotonsm-2hr-1
e = 0.012
Photon influx into Guard cells: 0.018mmolphotonsm-2hr-1
Photon influx into mesophyll cells: 485.982mmolphotonsm-2hr-1


## Run FBA without maintenance to get sum of fluxes

In [18]:
no_maintenance_solutions = pd.DataFrame()

### Blue Light Unconstrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        blue_unconstrained_wt,
        blue_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_uncon_wt'] = blue_unconstrained_wt_solution['fluxes']

### Blue Light Unconstrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        blue_unconstrained_ko,
        blue_unconstrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_uncon_ko'] = blue_unconstrained_ko_solution['fluxes']

### Blue Light Constrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    (
        blue_constrained_wt,
        blue_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_con_wt'] = blue_constrained_wt_solution['fluxes']

### Blue Light Constrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound) 
    (
        blue_constrained_ko,
        blue_constrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['blue_con_ko'] = blue_constrained_ko_solution['fluxes']

### White Light Unconstrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        white_unconstrained_wt,
        white_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_uncon_wt'] = white_unconstrained_wt_solution['fluxes']

### White Light Unconstrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        white_unconstrained_ko,
        white_unconstrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_uncon_ko'] = white_unconstrained_ko_solution['fluxes']

### White Light Constrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    (
        white_constrained_wt,
        white_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_con_wt'] = white_constrained_wt_solution['fluxes']

### White Light Constrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound) 
    (
        white_constrained_ko,
        white_constrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['white_con_ko'] = white_constrained_ko_solution['fluxes']

### No PS Unconstrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_uncon_wt'] = nops_unconstrained_wt_solution['fluxes']

### No PS Unconstrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound) 
    (
        nops_unconstrained_ko,
        nops_unconstrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_uncon_ko'] = nops_unconstrained_ko_solution['fluxes']

### No PS Constrained H+-ATPase WT

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)
    (
        nops_constrained_wt,
        nops_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_con_wt'] = nops_constrained_wt_solution['fluxes']

### No PS Constrained H+-ATPase Starch KO

arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    #m.reactions.Photon_tx_gc_2.upper_bound = 0
    #m.reactions.Photon_tx_me_2.upper_bound = 0
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound) 
    (
        nops_constrained_ko,
        nops_constrained_ko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=[], processes=no_processes)
no_maintenance_solutions['nops_con_ko'] = nops_constrained_ko_solution['fluxes']

Started running pFBA (and FVA) @ 2023-11-23 10:51:44.653878
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2023-11-23 10:51:56.974103, that took 0.20533707936604817 minutes
Started running pFBA (and FVA) @ 2023-11-23 10:51:56.986763
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2023-11-23 10:52:08.940485, that took 0.1992287000020345 minutes
Started running pFBA (and FVA) @ 2023-11-23 10:52:08.949169
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2023-11-23 10:52:20.446534, that took 0.19162275393803915 minutes
Started running pFBA (and FVA) @ 2023-11-23 10:52:20.455602
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2023-11-23 10:52:32.947294, that took 0.2081948717435201 minutes
Started running pFBA (and FVA) @ 2023-11-23 10:52:32.958987
Running pFBA
FVA list is empty
Finished running pFBA (and FVA) @ 2023-11-23 10:52:46.241495, that took 0.22137513160705566 minutes
Started running pFBA (and FVA) @ 2023-11-23 10:

### Get the list of reactions to exclude from sum of fluxes calculation

In [21]:
sum_fluxes_exclude = set()
for r in list(no_maintenance_solutions.index):
    parts = r.split('_')
    if parts[-1] in ['gc', 'me', 'day', 'overall', 'night']: # not in ['1', '2', '3', '4']
        sum_fluxes_exclude.add(r)
    else:
        if parts[-2] in ['tx', 'pseudolinker', 'Linker', 'a']: # not in ['me', 'gc']
            sum_fluxes_exclude.add(r)
        else:
            if parts[-3] in ['ae', 't', 'b', 'ep', 'tx', 'biomass']: # not in ['cv', 'p', 'xc', 'pc', 'mc', 'im', 'vc', 'mi', 'v', 'c', 'm', 'x', 'ec', 'pr', 'r', 'ce', 'cwINV']
                sum_fluxes_exclude.add(r)
            else:
                if 'constraint' in r:
                    sum_fluxes_exclude.add(r)

In [22]:
# Filter solution with only metabolic reactions and transporters (with constrained H+-ATPase only)
filtered_df = no_maintenance_solutions.loc[~no_maintenance_solutions.index.isin(sum_fluxes_exclude), no_maintenance_solutions.columns.str.contains('_con_')]

In [23]:
phase_length = {'1': 6, '2': 0.5, '3': 11.5, '4': 6}

def multiply_by_hours(row):
    index_suffix = row.name.split('_')[-1]
    factor = phase_length[index_suffix]
    return row * factor

scaled_df = filtered_df.apply(multiply_by_hours, axis=1)

#gc_sum_flux = filtered_df[filtered_df.index.str.contains('|'.join(['_gc_1', '_gc_4']))].abs().sum()
#me_sum_flux = filtered_df[filtered_df.index.str.contains('|'.join(['_me_1', '_me_4']))].abs().sum()
gc_sum_flux = scaled_df[scaled_df.index.str.contains('_gc_')].abs().sum()
me_sum_flux = scaled_df[scaled_df.index.str.contains('_me_')].abs().sum()

In [24]:
#print(scaled_df)
print(gc_sum_flux)
print(me_sum_flux)

blue_con_wt     0.901933
blue_con_ko     0.899520
white_con_wt    0.918848
white_con_ko    0.921276
nops_con_wt     0.201886
nops_con_ko     0.214510
dtype: float64
blue_con_wt     20142.122686
blue_con_ko     20142.122755
white_con_wt    20994.863369
white_con_ko    20994.863952
nops_con_wt     20994.859108
nops_con_ko     20994.859400
dtype: float64


In [27]:
gc_me_ratio = gc_sum_flux/me_sum_flux
print(gc_me_ratio)

blue_con_wt     0.000045
blue_con_ko     0.000045
white_con_wt    0.000044
white_con_ko    0.000044
nops_con_wt     0.000010
nops_con_ko     0.000010
dtype: float64


In [29]:
gc_me_ratio.head(4).mean()

4.4270867608601426e-05

## Add maintenance constraint to model

In [18]:
# | export

arabidopsis_supermodel.add_maintenance(gc=True, gc_ratio=4.4270867608601426e-05);

## Blue Light

### ATPase unconstrained

#### WT

In [31]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)
 
    (
        blue_unconstrained_wt,
        blue_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 10:55:56.119436
Running pFBA
Running FVA on 348 reactions using 6 processes
L_ASPARTATE_v_gc_Linker_2
RXN_10773_c_gc_2
SUC_Pi_mc_gc_2
L_ASPARTATE_xc_gc_2
ADENYL_KIN_RXN_c_gc_2
RXN490_3650_p_gc_2
RXN0_5184_c_gc_2
PROTON_ATPase_c_gc_2
CO2_ec_gc_2
MGII_ae_gc_3
Pi_ae_gc_3
AMMONIUM_ae_gc_3
TRP_v_gc_Linker_1
GLC_total_pseudolinker_1
FUMHYDR_RXN_c_gc_2
2KG_MAL_mc_gc_2
GLN_v_gc_Linker_1
NITRATE_v_gc_Linker_2
2KG_ACONITATE_mc_gc_2
MGII_ae_gc_4
Pi_ae_gc_4
AMMONIUM_ae_gc_4
PRO_v_gc_Linker_2
GLC_total_pseudolinker_2
Cl_cv_gc_2
TRP_v_gc_Linker_2
SUCROSE_v_gc_Linker_2
Photon_ep_gc_2
CIT_ACONITATE_mc_gc_2
GAPOXNPHOSPHN_RXN_p_gc_2
WATER_ae_gc_2
CAII_ae_gc_2
K_ae_gc_2
MAL_ae_gc_2
FRU_c_gc_Linker_1
FRU_PROTON_cv_gc_2
L_ASPARTATE_GLU_mc_gc_2
3PGAREARR_RXN_p_gc_2
PEPDEPHOS_RXN_c_gc_2
aCIT_v_gc_Linker_1
GLT_xc_gc_2
PHE_v_gc_Linker_2
Cl_c_gc_Linker_1
1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p_gc_2
CAII_ae_gc_1
MAL_ae_gc_1
K_ae_gc_1
WATER_ae_gc_1
PRO_v_gc_Linker_1
Cl_c_gc

In [33]:
# | export

blue_unconstrained_wt_solution.to_csv(output_dir + f"blue_unconstrained_wt.csv")

#### Starch KO

In [34]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        blue_unconstrained_starchko,
        blue_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 11:08:13.238050
Running pFBA
Running FVA on 348 reactions using 6 processes
aMAL_ae_gc_2
GLC_c_gc_Linker_1
MAL_SO4_mc_gc_2
ASN_v_gc_Linker_1
H_pc_gc_2
GLY_pc_gc_2
RIBULP3EPIM_RXN_c_gc_2
ASN_v_gc_Linker_2
GLC_c_gc_Linker_2
NITRATE_c_gc_Linker_1
aMAL_ae_gc_3
ISOCITRATE_ACONITATE_mc_gc_2
ASPAMINOTRANS_RXN_c_gc_2
SUC_SO4_mc_gc_2
K_c_gc_Linker_1
RXN_1781_v_gc_2
RXN_10786_c_gc_2
MAL_SUC_mc_gc_2
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
GLY_v_gc_Linker_1
aMAL_ae_gc_4
NITRATE_c_gc_Linker_2
ILE_v_gc_Linker_1
MAL_c_gc_Linker_1
GLY_v_gc_Linker_2
RXN_1827_p_gc_2
K_c_gc_Linker_2
K_v_gc_Linker_1
SEDOBISALDOL_RXN_p_gc_2
MET_v_gc_Linker_2
SUCROSE_c_gc_Linker_1
MAL_v_gc_Linker_2
Pi_PROTON_mc_gc_2
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi_gc_2
ILE_v_gc_Linker_2
2PGADEHYDRAT_RXN_c_gc_2
NADPHox_p_tx_gc_2
LYS_v_gc_Linker_1
OAA_SO4_mc_gc_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
K_v_gc_Linker_2
MET_v_gc_Linker_1
GLYOHMETRANS_RXN_p_gc_2
CITSYN_RXN_m_gc_2
FRU_c_gc_Li

In [38]:
# | export

blue_unconstrained_starchko_solution.to_csv(output_dir + "blue_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [39]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        blue_constrained_wt,
        blue_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 12:26:59.261477
Running pFBA
Running FVA on 348 reactions using 6 processes
PSII_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
GLU6PDEHYDROG_RXN_p_gc_2
6PGLUCONOLACT_RXN_p_gc_2
FRU_total_pseudolinker_2
MALIC_NAD_RXN_m_gc_2
ADP_pc_gc_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
DHAP_Pi_pc_gc_2
ASPAMINOTRANS_RXN_m_gc_2
GDPKIN_RXN_c_gc_2
2KG_SUC_mc_gc_2
6PFRUCTPHOS_RXN_p_gc_2
K_total_pseudolinker_1
PROTONATP_rev_vc_gc_2
K_total_pseudolinker_2
2TRANSKETO_RXN_p_gc_2
PEPCARBOX_RXN_c_gc_2
OAA_MAL_pc_gc_2
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
2PGADEHYDRAT_RXN_p_gc_2
6PGLUCONDEHYDROG_RXN_c_gc_2
ATP_pc_gc_2
ACONITATEHYDR_RXN_c_gc_2
Cl_total_pseudolinker_1
SUCROSE_cv_gc_2
PHOSGLYPHOS_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
Cl_total_pseudolinker_2
NITRATE_ae_gc_1
2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2
ATP_AMP_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
PROTON_PPi_rev_vc_gc_2
NITRATE_ae_gc_2
OAA_xc_gc_2
GLC_pc_gc_2
2KG_CIT_mc_gc_2
1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_

In [40]:
# | export

blue_constrained_wt_solution.to_csv(output_dir + "blue_constrained_wt.csv")

#### Starch KO

In [41]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    m.reactions.Photon_tx_gc_2.upper_bound = 0
    m.reactions.Photon_tx_me_2.upper_bound = 0
    
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        blue_constrained_starchko,
        blue_constrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 13:09:32.629978
Running pFBA
Running FVA on 348 reactions using 6 processes
PSII_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
GLU6PDEHYDROG_RXN_p_gc_2
6PGLUCONOLACT_RXN_p_gc_2
FRU_total_pseudolinker_2
MALIC_NAD_RXN_m_gc_2
ADP_pc_gc_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
DHAP_Pi_pc_gc_2
ASPAMINOTRANS_RXN_m_gc_2
GDPKIN_RXN_c_gc_2
2KG_SUC_mc_gc_2
6PFRUCTPHOS_RXN_p_gc_2
K_total_pseudolinker_1
PROTONATP_rev_vc_gc_2
K_total_pseudolinker_2
2TRANSKETO_RXN_p_gc_2
PEPCARBOX_RXN_c_gc_2
OAA_MAL_pc_gc_2
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
2PGADEHYDRAT_RXN_p_gc_2
6PGLUCONDEHYDROG_RXN_c_gc_2
ATP_pc_gc_2
ACONITATEHYDR_RXN_c_gc_2
Cl_total_pseudolinker_1
SUCROSE_cv_gc_2
PHOSGLYPHOS_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
Cl_total_pseudolinker_2
NITRATE_ae_gc_1
2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2
ATP_AMP_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
PROTON_PPi_rev_vc_gc_2
NITRATE_ae_gc_2
OAA_xc_gc_2
GLC_pc_gc_2
2KG_CIT_mc_gc_2
1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_

In [42]:
# | export

blue_constrained_starchko_solution.to_csv(output_dir + "blue_constrained_starchko.csv")

## White Light

### Unconstrained ATPase

#### WT

In [43]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        white_unconstrained_wt,
        white_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 13:25:32.637918
Running pFBA
Running FVA on 348 reactions using 6 processes
L_ASPARTATE_v_gc_Linker_2
RXN_10773_c_gc_2
SUC_Pi_mc_gc_2
L_ASPARTATE_xc_gc_2
ADENYL_KIN_RXN_c_gc_2
RXN490_3650_p_gc_2
RXN0_5184_c_gc_2
PROTON_ATPase_c_gc_2
CO2_ec_gc_2
MGII_ae_gc_3
Pi_ae_gc_3
AMMONIUM_ae_gc_3
TRP_v_gc_Linker_1
GLC_total_pseudolinker_1
FUMHYDR_RXN_c_gc_2
2KG_MAL_mc_gc_2
GLN_v_gc_Linker_1
NITRATE_v_gc_Linker_2
2KG_ACONITATE_mc_gc_2
MGII_ae_gc_4
Pi_ae_gc_4
AMMONIUM_ae_gc_4
PRO_v_gc_Linker_2
GLC_total_pseudolinker_2
Cl_cv_gc_2
TRP_v_gc_Linker_2
SUCROSE_v_gc_Linker_2
Photon_ep_gc_2
CIT_ACONITATE_mc_gc_2
GAPOXNPHOSPHN_RXN_p_gc_2
WATER_ae_gc_2
CAII_ae_gc_2
K_ae_gc_2
MAL_ae_gc_2
FRU_c_gc_Linker_1
FRU_PROTON_cv_gc_2
L_ASPARTATE_GLU_mc_gc_2
3PGAREARR_RXN_p_gc_2
PEPDEPHOS_RXN_c_gc_2
aCIT_v_gc_Linker_1
GLT_xc_gc_2
PHE_v_gc_Linker_2
Cl_c_gc_Linker_1
1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p_gc_2
CAII_ae_gc_1
MAL_ae_gc_1
K_ae_gc_1
WATER_ae_gc_1
PRO_v_gc_Linker_1
Cl_c_gc

In [45]:
# | export

white_unconstrained_wt_solution.to_csv(output_dir + "white_unconstrained_wt.csv")

#### Starch KO

In [46]:
 # | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        white_unconstrained_starchko,
        white_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 15:51:21.839118
Running pFBA
Running FVA on 348 reactions using 6 processes
L_ASPARTATE_v_gc_Linker_2
RXN_10773_c_gc_2
SUC_Pi_mc_gc_2
L_ASPARTATE_xc_gc_2
ADENYL_KIN_RXN_c_gc_2
RXN490_3650_p_gc_2
RXN0_5184_c_gc_2
PROTON_ATPase_c_gc_2
CO2_ec_gc_2
MGII_ae_gc_3
Pi_ae_gc_3
AMMONIUM_ae_gc_3
TRP_v_gc_Linker_1
GLC_total_pseudolinker_1
FUMHYDR_RXN_c_gc_2
2KG_MAL_mc_gc_2
GLN_v_gc_Linker_1
NITRATE_v_gc_Linker_2
2KG_ACONITATE_mc_gc_2
MGII_ae_gc_4
Pi_ae_gc_4
AMMONIUM_ae_gc_4
PRO_v_gc_Linker_2
GLC_total_pseudolinker_2
Cl_cv_gc_2
TRP_v_gc_Linker_2
SUCROSE_v_gc_Linker_2
Photon_ep_gc_2
CIT_ACONITATE_mc_gc_2
GAPOXNPHOSPHN_RXN_p_gc_2
WATER_ae_gc_2
CAII_ae_gc_2
K_ae_gc_2
MAL_ae_gc_2
FRU_c_gc_Linker_1
FRU_PROTON_cv_gc_2
L_ASPARTATE_GLU_mc_gc_2
3PGAREARR_RXN_p_gc_2
PEPDEPHOS_RXN_c_gc_2
aCIT_v_gc_Linker_1
GLT_xc_gc_2
PHE_v_gc_Linker_2
Cl_c_gc_Linker_1
1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p_gc_2
CAII_ae_gc_1
MAL_ae_gc_1
K_ae_gc_1
WATER_ae_gc_1
PRO_v_gc_Linker_1
Cl_c_gc

In [47]:
# | export

white_unconstrained_starchko_solution.to_csv(output_dir + "white_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [48]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        white_constrained_wt,
        white_constrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 16:39:54.776365
Running pFBA
Running FVA on 348 reactions using 6 processes
PSII_RXN_p_gc_2
MALATE_DEH_RXN_c_gc_2
GLU6PDEHYDROG_RXN_p_gc_2
6PGLUCONOLACT_RXN_p_gc_2
FRU_total_pseudolinker_2
MALIC_NAD_RXN_m_gc_2
ADP_pc_gc_2
PHOSPHORIBULOKINASE_RXN_p_gc_2
DHAP_Pi_pc_gc_2
ASPAMINOTRANS_RXN_m_gc_2
GDPKIN_RXN_c_gc_2
2KG_SUC_mc_gc_2
6PFRUCTPHOS_RXN_p_gc_2
K_total_pseudolinker_1
PROTONATP_rev_vc_gc_2
K_total_pseudolinker_2
2TRANSKETO_RXN_p_gc_2
PEPCARBOX_RXN_c_gc_2
OAA_MAL_pc_gc_2
SUCCINATE_DEHYDROGENASE_UBIQUINONE_RXN_mi_gc_2
2PGADEHYDRAT_RXN_p_gc_2
6PGLUCONDEHYDROG_RXN_c_gc_2
ATP_pc_gc_2
ACONITATEHYDR_RXN_c_gc_2
Cl_total_pseudolinker_1
SUCROSE_cv_gc_2
PHOSGLYPHOS_RXN_c_gc_2
1TRANSKETO_RXN_p_gc_2
Cl_total_pseudolinker_2
NITRATE_ae_gc_1
2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2
ATP_AMP_mc_gc_2
CELLULOSE_SYNTHASE_GDP_FORMING_RXN_c_gc_2
PROTON_PPi_rev_vc_gc_2
NITRATE_ae_gc_2
OAA_xc_gc_2
GLC_pc_gc_2
2KG_CIT_mc_gc_2
1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p_gc_

In [49]:
# | export

white_constrained_wt_solution.to_csv(output_dir + "white_constrained_wt.csv")

#### Starch KO

In [19]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        white_constrained_starchko,
        white_constrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 18:56:58.728546
Running pFBA
Running FVA on 348 reactions using 6 processes
NITRATE_ae_gc_3
ATP_AMP_mc_gc_2
RIBULP3EPIM_RXN_c_gc_2
CIT_total_pseudolinker_2
SUC_Pi_mc_gc_2
NADPHox_p_tx_gc_2
GLC_PROTON_rev_cv_gc_2
GLY_v_gc_Linker_1
CIT_total_pseudolinker_1
NITRATE_ae_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
3PGAREARR_RXN_c_gc_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
MAL_total_pseudolinker_1
OAA_xc_gc_2
ACONITATEDEHYDR_RXN_c_gc_2
MAL_total_pseudolinker_2
THF_pc_gc_2
PEPDEPHOS_RXN_p_gc_2
2KG_OAA_mc_gc_2
MAL_ACONITATE_mc_gc_2
FRU_total_pseudolinker_1
SUCROSE_PHOSPHATASE_RXN_c_gc_2
FRU_total_pseudolinker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
PGLUCISOM_RXN_p_gc_2
F16BDEPHOS_RXN_p_gc_2
NADPH_Dehydrogenase_p_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_2
FRU_c_gc_Linker_1
ADENYL_KIN_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_p_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
FRUCTOKINASE_RXN_c_gc_2
K_total_pseudolinker_1
NITRATE_ae_gc_1
GL

In [20]:
# | export

white_constrained_starchko_solution.to_csv(output_dir + "white_constrained_starchko.csv")

## Guard cell has no photosynthetic capacity

### Unconstrained ATPase

#### WT

In [21]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-23 19:03:54.651962
Running pFBA
Running FVA on 348 reactions using 6 processes
NITRATE_ae_gc_3
ATP_AMP_mc_gc_2
RIBULP3EPIM_RXN_c_gc_2
CIT_total_pseudolinker_2
SUC_Pi_mc_gc_2
NADPHox_p_tx_gc_2
GLC_PROTON_rev_cv_gc_2
GLY_v_gc_Linker_1
CIT_total_pseudolinker_1
NITRATE_ae_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
3PGAREARR_RXN_c_gc_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
MAL_total_pseudolinker_1
OAA_xc_gc_2
ACONITATEDEHYDR_RXN_c_gc_2
MAL_total_pseudolinker_2
THF_pc_gc_2
PEPDEPHOS_RXN_p_gc_2
2KG_OAA_mc_gc_2
MAL_ACONITATE_mc_gc_2
FRU_total_pseudolinker_1
SUCROSE_PHOSPHATASE_RXN_c_gc_2
FRU_total_pseudolinker_2
K_ec_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_1
PGLUCISOM_RXN_p_gc_2
F16BDEPHOS_RXN_p_gc_2
NADPH_Dehydrogenase_p_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_2
FRU_c_gc_Linker_1
ADENYL_KIN_RXN_p_gc_2
GAPOXNPHOSPHN_RXN_p_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
FRUCTOKINASE_RXN_c_gc_2
K_total_pseudolinker_1
NITRATE_ae_gc_1
GL

In [22]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_unconstrained_wt.csv")

#### Starch KO

In [19]:
# | export
# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    # Added this line to explictly set bounds for H+-ATPase in GC
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, cobra.Configuration().upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)
    
    (
        nops_unconstrained_starchko,
        nops_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-27 14:11:49.953485
Running pFBA
Running FVA on 348 reactions using 6 processes
GLY_v_gc_Linker_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
ILE_v_gc_Linker_2
MET_v_gc_Linker_1
K_c_gc_Linker_1
RXN_1827_p_gc_2
MAL_total_pseudolinker_2
WATER_vc_gc_2
K_c_gc_Linker_2
2KG_MAL_mc_gc_2
VAL_v_gc_Linker_1
2KG_SUC_mc_gc_2
PROTONATP_rev_vc_gc_2
GLUCOKIN_RXN_c_gc_2
FRU_total_pseudolinker_1
NADPH_Dehydrogenase_p_gc_2
VAL_v_gc_Linker_2
SEDOBISALDOL_RXN_p_gc_2
FRU_total_pseudolinker_2
LYS_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_1
OAA_MAL_mc_gc_2
SUCCCOASYN_RXN_m_gc_2
ASPAMINOTRANS_RXN_x_gc_2
GLT_xc_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_2
LYS_v_gc_Linker_2
FRU_c_gc_Linker_1
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
K_total_pseudolinker_1
ILE_v_gc_Linker_1
MAL_c_gc_Linker_1
GAP_Pi_pc_gc_2
SUC_ISOCITRATE_mc_gc_2
L_ALPHA_ALANINE_v_gc_Linker_1
K_total_pseudolinker_2
FRU_c_gc_Linker_2
SUC_Pi_mc_gc_2
K_PROTON_cv_gc_2
PLASTOQUINOL_PL

/home/maurice/miniconda3/envs/mmon-gcm/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


FRU_ae_gc_2
ACONITATEHYDR_RXN_m_gc_2
MAL_v_gc_Linker_2
MALATE_DEH_RXN_c_gc_2
SUCROSE_PROTON_ec_gc_2
RXN_1461_v_gc_2
METHYLENE_THF_pc_gc_2
RXN_1841_v_gc_2
MANNPGUANYLTRANGDP_RXN_c_gc_2
OAA_ISOCITRATE_mc_gc_2
RXN_10786_c_gc_2
2TRANSKETO_RXN_p_gc_2
Cl_ae_gc_3
FRU_ae_gc_3
GLC_PROTON_ec_gc_2
ARG_v_gc_Linker_1
CIT_ISOCITRATE_mc_gc_2
PEPCARBOXYKIN_RXN_c_gc_2
OAA_CIT_mc_gc_2
MGII_ae_gc_4
Pi_ae_gc_4
aMAL_ae_gc_4
Cl_ae_gc_4
FRU_ae_gc_4
MALIC_NADP_RXN_p_gc_2
ARG_v_gc_Linker_2
PALMITATE_c_gc_Linker_2
ATP_pc_gc_2
HIS_v_gc_Linker_1
ACONITATEDEHYDR_RXN_m_gc_2
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
F16ALDOLASE_RXN_p_gc_2
MALATE_DEH_RXN_m_gc_2
HIS_v_gc_Linker_2
SUCROSE_total_pseudolinker_1
MGII_ae_gc_1
Pi_ae_gc_1
aMAL_ae_gc_1
MAL_ae_gc_1
2_PERIOD_7_PERIOD_7_PERIOD_13_RXN_c_gc_2
aMAL_ae_gc_2
MGII_ae_gc_2
Pi_ae_gc_2
SUCROSE_total_pseudolinker_2
G6P_Pi_pc_gc_2
2_PERIOD_7_PERIOD_7_PERIOD_34_RXN_c_gc_2
MGII_ae_gc_3
Pi_ae_gc_3
aMAL_ae_gc_3
bHIS_v_gc_Linker_1
NITRATE_v_gc_Linker_1
GLC_total_pseudolinker

In [20]:
# | export

nops_unconstrained_starchko_solution.to_csv(output_dir + "nops_unconstrained_starchko.csv")

### Constrained ATPase

#### WT

In [21]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    (
        nops_unconstrained_wt,
        nops_unconstrained_wt_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-27 14:18:19.978287
Running pFBA
Running FVA on 348 reactions using 6 processes
GLY_v_gc_Linker_2
SUCCINATE_COA_LIGASE_GDP_FORMING_RXN_m_gc_2
ILE_v_gc_Linker_2
MET_v_gc_Linker_1
K_c_gc_Linker_1
RXN_1827_p_gc_2
MAL_total_pseudolinker_2
WATER_vc_gc_2
K_c_gc_Linker_2
2KG_MAL_mc_gc_2
VAL_v_gc_Linker_1
2KG_SUC_mc_gc_2
PROTONATP_rev_vc_gc_2
GLUCOKIN_RXN_c_gc_2
FRU_total_pseudolinker_1
NADPH_Dehydrogenase_p_gc_2
VAL_v_gc_Linker_2
SEDOBISALDOL_RXN_p_gc_2
FRU_total_pseudolinker_2
LYS_v_gc_Linker_1
4_AMINO_BUTYRATE_v_gc_Linker_1
OAA_MAL_mc_gc_2
SUCCCOASYN_RXN_m_gc_2
ASPAMINOTRANS_RXN_x_gc_2
GLT_xc_gc_2
4_AMINO_BUTYRATE_v_gc_Linker_2
LYS_v_gc_Linker_2
FRU_c_gc_Linker_1
CONIFERIN_BETA_GLUCOSIDASE_RXN_c_gc_2
SEDOHEPTULOSE_BISPHOSPHATASE_RXN_p_gc_2
K_total_pseudolinker_1
ILE_v_gc_Linker_1
MAL_c_gc_Linker_1
GAP_Pi_pc_gc_2
SUC_ISOCITRATE_mc_gc_2
L_ALPHA_ALANINE_v_gc_Linker_1
K_total_pseudolinker_2
FRU_c_gc_Linker_2
SUC_Pi_mc_gc_2
K_PROTON_cv_gc_2
PLASTOQUINOL_PL

In [22]:
# | export

nops_unconstrained_wt_solution.to_csv(output_dir + "nops_constrained_wt.csv")

#### Starch KO

In [19]:
# | export

# Added this line to explictly set Photon constraint
arabidopsis_supermodel.constrain_photons(PPFD, printouts=False)
with arabidopsis_supermodel.fba_model as m:
    for p in [1, 2, 3, 4]:
        m.reactions.get_by_id(f"Photon_tx_gc_{p}").bounds = (0, 0)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, 0)

    (
        nops_unconstrained_starchko,
        nops_unconstrained_starchko_solution,
    ) = mmon_gcm.solving.get_pfba_fva_solution(m, rxn_list=fva_list, processes=no_processes)

Started running pFBA (and FVA) @ 2023-11-27 14:49:50.209009
Running pFBA
Running FVA on 348 reactions using 6 processes
MANNPGUANYLTRANGDP_RXN_c_gc_2
K_total_pseudolinker_2
FRU_c_gc_Linker_2
K_PROTON_cv_gc_2
2KG_CIT_mc_gc_2
PLASTOQUINOL_PLASTOCYANIN_REDUCTASE_RXN_p_gc_2
SUC_Pi_mc_gc_2
GLY_v_gc_Linker_1
ILE_v_gc_Linker_1
Cl_total_pseudolinker_2
LEU_v_gc_Linker_1
THR_v_gc_Linker_2
GLC_c_gc_Linker_1
RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_gc_2
ACONITATEHYDR_RXN_c_gc_2
THR_v_gc_Linker_1
Cl_total_pseudolinker_1
1TRANSKETO_RXN_p_gc_2
PHOSGLYPHOS_RXN_c_gc_2
2OXOGLUTARATEDEH_RXN_m_gc_2
FRU_PROTON_ec_gc_2
H2O_ec_gc_2
NITRATE_ae_gc_2
ADENYL_KIN_RXN_m_gc_2
3PGA_Pi_pc_gc_2
H2O_pc_gc_2
PEP_Pi_pc_gc_2
ASN_v_gc_Linker_1
NITRATE_c_gc_Linker_2
MALATE_DEHYDROGENASE_NADPs_RXN_p_gc_2
NITRATE_ae_gc_1
ASN_v_gc_Linker_2
CELLULOSE_SYNTHASE_UDP_FORMING_RXN_c_gc_2
CIT_total_pseudolinker_2
SUC_CIT_mc_gc_2
L_ASPARTATE_v_gc_Linker_1
Cl_v_gc_Linker_1
NITRATE_ae_gc_4
MGII_ae_gc_4
Pi_ae_gc_4
NITRATE_c_gc_Linker_1
6PF

In [20]:
# | export

nops_unconstrained_starchko_solution.to_csv(output_dir + "nops_constrained_starchko.csv")

## Scan GC photon influx to look at switch of glycolysis

In [19]:
P_gc, P_me = arabidopsis_supermodel.get_photons(PPFD=PPFD, printouts=False)

for i, p in enumerate([0, P_me, P_me, 0], start=1):
    arabidopsis_supermodel.fba_model.reactions.get_by_id("Photon_tx_me_" + str(i)).bounds = 0, p

with arabidopsis_supermodel.fba_model as m:
    # Added this line to explictly set bounds for starch degradation
    mmon_gcm.buildingediting.set_bounds_multi(m, "RXN_1827_p_gc", 0, cobra.Configuration().upper_bound)

    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.5)
    mmon_gcm.buildingediting.set_bounds_multi(m, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound)

    scan_df = pd.DataFrame()
    for photon in range(PPFD+1):
        P_gc, P_me = arabidopsis_supermodel.get_photons(PPFD=photon, printouts=False)
        for i, p in enumerate([0, P_gc, P_gc, 0], start=1):
            m.reactions.get_by_id("Photon_tx_gc_" + str(i)).bounds = 0, p
        cobra_model, solution = mmon_gcm.solving.pfba_Weighted(m, objective="Phloem_tx_overall")
        scan_df[photon] = solution.fluxes
        print(photon, solution.fluxes['Photon_tx_gc_2'], solution.fluxes['Photon_tx_me_2'])
scan_df.to_csv("../outputs/scans/gc_photon_scan.csv", )

0 0.0 485.98162812525
1 0.000122479164998293 485.98162812525
2 0.000244958329996587 485.98162812525
3 0.00036743749499488 485.98162812525
4 0.000489916659993173 485.98162812525
5 0.000612395824991467 485.98162812525
6 0.00073487498998976 485.98162812525
7 0.000857354154988053 485.98162812525
8 0.000979833319986346 485.98162812525
9 0.00110231248498464 485.98162812525
10 0.00122479164998293 485.98162812525
11 0.00134727081498123 485.98162812525
12 0.00146974997997952 485.98162812525
13 0.00159222914497781 485.98162812525
14 0.00171470830997611 485.98162812525
15 0.0018371874749744 485.98162812525
16 0.00195966663997269 485.98162812525
17 0.00208214580497099 485.98162812525
18 0.00220462496996928 485.98162812525
19 0.00232710413496757 485.98162812525
20 0.00244958329996587 485.98162812525
21 0.00257206246496416 485.98162812525
22 0.00269454162996245 485.98162812525
23 0.00281702079496075 485.98162812525
24 0.00293949995995904 485.98162812525
25 0.00306197912495733 485.98162812525
26 0.00

/tmp/ipykernel_8788/972194867.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


100 0.0122479164998293 485.98162812525


/tmp/ipykernel_8788/972194867.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


101 0.0123703956648276 485.98162812525


...

148 0.0181269164197474 485.98162812525


/tmp/ipykernel_8788/972194867.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


149 0.0182493955847457 485.98162812525


/tmp/ipykernel_8788/972194867.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scan_df[photon] = solution.fluxes


150 0.018371874749744 485.98162812525
